In [2]:
from reservoirpy.nodes import Reservoir, Identity
import numpy as np

nb_neurons = 50
steps = 50

# Create a reservoir with nb_neurons
reservoir = Reservoir(units=nb_neurons, sr=0.9, lr=0.3)
readout = Identity()
reservoir <<= readout
model = reservoir >> readout

# Initialize
first_step = np.zeros((1, nb_neurons))
sample_played = []
penalty = 1
penalty_length = 5

for i in range(50):
    # Run the model
    output = model.call(first_step)

    # Apply penalty
    output[0, sample_played] -= penalty

    # Get the sample
    sample = np.argmax(output[0])
    print(f"Sample {i}: {sample}")

    # Update the sample played
    sample_played.append(sample)
    if len(sample_played) > penalty_length:
        sample_played.pop(0)

Sample 0: 2
Sample 1: 17
Sample 2: 34
Sample 3: 28
Sample 4: 16
Sample 5: 49
Sample 6: 2
Sample 7: 17
Sample 8: 42
Sample 9: 28
Sample 10: 21
Sample 11: 49
Sample 12: 2
Sample 13: 17
Sample 14: 42
Sample 15: 28
Sample 16: 3
Sample 17: 49
Sample 18: 46
Sample 19: 17
Sample 20: 42
Sample 21: 28
Sample 22: 30
Sample 23: 49
Sample 24: 46
Sample 25: 17
Sample 26: 42
Sample 27: 28
Sample 28: 39
Sample 29: 49
Sample 30: 46
Sample 31: 17
Sample 32: 42
Sample 33: 28
Sample 34: 39
Sample 35: 49
Sample 36: 46
Sample 37: 17
Sample 38: 42
Sample 39: 28
Sample 40: 43
Sample 41: 49
Sample 42: 46
Sample 43: 17
Sample 44: 41
Sample 45: 28
Sample 46: 43
Sample 47: 49
Sample 48: 2
Sample 49: 17
